In [1]:
from data import get_tables
from data import get_embedding_sizes
from data import predict_and_save
from fastai.structured import *
from fastai.column_data import ColumnarModelData
np.set_printoptions(threshold=50, edgeitems=20)

In [2]:
tables = get_tables('data/', ['train', 'test'])

# Not using last name or cabin right now - cardinality is too high
cat_vars = ['Pclass', 'Sex', 'Embarked', 'Title']
cont_vars = ['Age', 'SibSp', 'Parch', 'Fare']

for table in tables:
    for v in cat_vars:
        table[v] = table[v].astype('category').cat.as_ordered()

train, test = tables
test['Survived'] = 0
train = train[cat_vars + cont_vars + ['Survived']]
test = test[cat_vars + cont_vars + ['Survived', 'PassengerId']]

df, y, nas, mapper = proc_df(train, 'Survived', do_scale=True)
df_test, _, nas, mapper = proc_df(test, 'Survived', do_scale=True, skip_flds=['PassengerId'],
                                  mapper=mapper, na_dict=nas)

In [3]:
val_idx = df.sample(frac=0.25).index

md = ColumnarModelData.from_data_frame('data/', val_idx, df, y.astype(np.float32), cat_flds=cat_vars,
                                       bs=64, test_df=df_test)
embedding_sizes = get_embedding_sizes(cat_vars, train)

model = md.get_learner(embedding_sizes, len(df.columns) - len(cat_vars), 0.04, 1, [8, 4], [0.001, 0.01],
                       y_range=(0, 1))
#model.summary()


In [4]:
#model.fit(1e-3, 80)
#model.save('model-0')

In [5]:
model.load('m-1')

In [6]:
model.fit(1e-4, 30)

epoch      trn_loss   val_loss                                                                                                                                                                                          
    0      0.110474   0.137816  
    1      0.111191   0.137922                                                                                                                                                                                          
    2      0.113721   0.137872                                                                                                                                                                                          
    3      0.117034   0.137912                                                                                                                                                                                          
    4      0.117877   0.137997                                                                     

[array([0.13802])]

In [7]:
predict_and_save(model, test, 'sub-1')

c:\users\yingx\miniconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
